## Notebook for Session 9

**Note:** This notebook is to be seen as a 'scratch-pad', illustrating the points made in the slides.

In [2]:
import json
import pandas as pd
import numpy as np
import os
import sys
import tensorflow as tf
#import tensorflow_hub as hub
from time import time

import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.ticker import PercentFormatter

from tensorflow.keras import backend as K
from tensorflow.keras import layers
from tensorflow.python.keras.layers import Lambda, Dense
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.backend import sparse_categorical_crossentropy

### 1. Generation of Some Synthetic Test Data 

Imagine 400 train and 400 test examples with 10 features each. Each example has **two** labels, label 'a' and label 'b'. We construct these examples synthetically by picking 10 random numbers between 0 and 1, and adding for the examples with label $a =  1$ 0.2 to the first five features, and subtracting 0.2 in those features for examples with label $a = 0$. We do the same for label $b$, where we add/subtract 0.1 to features 6-10. (Obviously, this is just a toy example and the details do not matter - we just want to have some data to explore the formalisms.)

In [3]:
train_1_X = np.array([0.2] * 5 + [0.1] * 5) + np.reshape(np.random.random(1000),[100,10])
train_2_X = np.array([0.2] * 5 + [-0.1] * 5) + np.reshape(np.random.random(1000),[100,10])
train_3_X = np.array([-0.2] * 5 + [0.1] * 5) + np.reshape(np.random.random(1000),[100,10])
train_4_X = np.array([-0.2] * 5 + [-0.1] * 5) + np.reshape(np.random.random(1000),[100,10])

train_X = np.concatenate([train_1_X, train_2_X, train_3_X, train_4_X])

test_1_X = np.array([0.2] * 5 + [0.1] * 5) + np.reshape(np.random.random(1000),[100,10])
test_2_X = np.array([0.2] * 5 + [-0.1] * 5) + np.reshape(np.random.random(1000),[100,10])
test_3_X = np.array([-0.2] * 5 + [0.1] * 5) + np.reshape(np.random.random(1000),[100,10])
test_4_X = np.array([-0.2] * 5 + [-0.1] * 5) + np.reshape(np.random.random(1000),[100,10])

test_X = np.concatenate([test_1_X, test_2_X, test_3_X, test_4_X])

train_1a_y = train_1b_y = train_2a_y = train_3b_y = np.reshape([[1] * 100], [100,1])
train_2b_y = train_3a_y = train_4a_y = train_4b_y = np.reshape([[0] * 100], [100,1])

test_1a_y = test_1b_y = test_2a_y = test_3b_y = np.reshape([[1] * 100], [100,1])
test_2b_y = test_3a_y = test_4a_y = test_4b_y = np.reshape([[0] * 100], [100,1])

train_a_y = np.concatenate([train_1a_y, train_2a_y, train_3a_y, train_4a_y])
train_b_y = np.concatenate([train_1b_y, train_2b_y, train_3b_y, train_4b_y])

test_a_y = np.concatenate([test_1a_y, test_2a_y, test_3a_y, test_4a_y])
test_b_y = np.concatenate([test_1b_y, test_2b_y, test_3b_y, test_4b_y])


Shapes as expected?

In [4]:
np.shape(train_X)

(400, 10)

In [5]:
train_X[:2,:]

array([[0.20090649, 0.5368248 , 0.75112714, 0.37645579, 1.01529157,
        0.38071727, 0.60069459, 0.96073541, 1.06558354, 1.04067819],
       [0.96083078, 0.44315406, 0.70021697, 0.47023007, 0.8673389 ,
        0.9461193 , 0.25646862, 1.05648187, 0.34222064, 0.87750498]])

In [6]:
np.shape(train_a_y)

(400, 1)

In [7]:
train_a_y[:2, :]

array([[1],
       [1]])

Looks good.

### 2. Keras' Functional API Formalism

Keras' Functional API allows us to construct non-sequential models with multiple inputs, multiple outputs, branched models,  etc..

Key aspects:

* start with definition of inputs through Input layer(s)
* define the layers and outputs as:
$$\rm next\_layer\_activations = layer(layer\_parameters) (previous\_layer\_activations)$$
* define the model the the input(s) and output(s).  
* 'compile' the model by specifying loss function(s), metric(s) and the optimizer.  

We first set things up for one binary classification. It is most convenient to define a function called *build_model()* where we can parametrize aspects we want to vary.

In [8]:
def build_model(activation_function='relu', 
                optimizer='adam', 
                kernel_initializer=tf.keras.initializers.he_normal(seed=1), 
                bias_initializer='zeros'):

    # Define Input layer(s)
    input_numbers = tf.keras.layers.Input(shape=(10,), name="input_numbers")

    # Define one hidden layer and act on input
    dense_layer = tf.keras.layers.Dense(10, 
                                        activation=activation_function, 
                                        kernel_initializer=kernel_initializer, 
                                        bias_initializer=bias_initializer,
                                        name='dense') # layer definition
    dense_output = dense_layer(input_numbers)  # layer acting on previous layer's output
    
    # Define one hidden layer and act on input
    dense_layer_2 = tf.keras.layers.Dense(10, 
                                        activation=activation_function, 
                                        kernel_initializer=kernel_initializer, 
                                        bias_initializer=bias_initializer,
                                        name='dense_2') # layer definition
    dense_output_2 = dense_layer_2(dense_output)  # layer acting on previous layer's output

    

    # Define classification layer and act on previous output to classify examples
    classification_layer = tf.keras.layers.Dense(1, activation='sigmoid', name='classification') # layer definition
    classification_output = classification_layer(dense_output_2)   # layer acting on previous layer's output

    # Build and compile model
    model = tf.keras.models.Model(input_numbers, classification_output)
    
    model.compile(loss='binary_crossentropy', optimizer=optimizer)
    
    return model

model = build_model()


model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_numbers (InputLayer)   [(None, 10)]              0         
_________________________________________________________________
dense (Dense)                (None, 10)                110       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                110       
_________________________________________________________________
classification (Dense)       (None, 1)                 11        
Total params: 231
Trainable params: 231
Non-trainable params: 0
_________________________________________________________________


Easy!   

**Question:** Do the dimensions look right? 

### 3.  Training Optimizations

Great, let's do some experimenting. We run the model above for the synthetic data set we defined, only considering the first label, and look at a few variations. Specifically, let's play with:

* the **activation function** in the hidden layer     
* the **initialization**    
* the **optimizer**

We run everything for 40 epochs and compare the results. (We split the training into two parts to cut down on reporting. 'verbose=0' supresses output. But we still want to see the final loss so we add one epoch with 'verbose=1'.)

In [9]:
model_1 = build_model()  # This one should be good...

model_1.fit(
        train_X,
        train_a_y,
        validation_data=[test_X, test_a_y],
        epochs=39,
        verbose=0)

model_1.fit(
        train_X,
        train_a_y,
        validation_data=[test_X, test_a_y],
        epochs=1,
        verbose=1)

Train on 400 samples, validate on 400 samples
400/400 [==============================] - 0s 60us/sample - loss: 0.2285 - val_loss: 0.1856


This is our supposed best setup, and it defines our baselines. 

Let's veer off and see how loss, etc. change. First, we use $tanh()$ as opposed to $relu()$:

In [10]:
model_2 = build_model(activation_function='tanh')  

model_2.fit(
        train_X,
        train_a_y,
        validation_data=[test_X, test_a_y],
        epochs=39,
        verbose=0)

model_2.fit(
        train_X,
        train_a_y,
        validation_data=[test_X, test_a_y],
        epochs=1,
        verbose=1)

Train on 400 samples, validate on 400 samples
400/400 [==============================] - 0s 61us/sample - loss: 0.2601 - val_loss: 0.2040


A bit worse. [Of course, the initialization is random, so things may be different on future runs.]

Next, we change the optimizer to stochastic gradient descent:

In [11]:
model_3 = build_model(optimizer='sgd')  

model_3.fit(
        train_X,
        train_a_y,
        validation_data=[test_X, test_a_y],
        epochs=39,
        verbose=0)

model_3.fit(
        train_X,
        train_a_y,
        validation_data=[test_X, test_a_y],
        epochs=1,
        verbose=1)

Train on 400 samples, validate on 400 samples
400/400 [==============================] - 0s 62us/sample - loss: 0.3979 - val_loss: 0.3742


A lot worse!!

Going back to the activation function, what about $sigmoid()$?

In [12]:
model_4 = build_model(activation_function='sigmoid')  

model_4.fit(
        train_X,
        train_a_y,
        validation_data=[test_X, test_a_y],
        epochs=39,
        verbose=0)

model_4.fit(
        train_X,
        train_a_y,
        validation_data=[test_X, test_a_y],
        epochs=1,
        verbose=1)

Train on 400 samples, validate on 400 samples
400/400 [==============================] - 0s 62us/sample - loss: 0.4683 - val_loss: 0.4538


Not good!

Lastly, what about choosing general random matrix initialization as opposed to He/Xavier? 

In [13]:
model_5 = build_model(kernel_initializer='random_normal')  

model_5.fit(
        train_X,
        train_a_y,
        validation_data=[test_X, test_a_y],
        epochs=39,
        verbose=0)

model_5.fit(
        train_X,
        train_a_y,
        validation_data=[test_X, test_a_y],
        epochs=1,
        verbose=1)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Train on 400 samples, validate on 400 samples
400/400 [==============================] - 0s 63us/sample - loss: 0.1937 - val_loss: 0.1496


A bit worse too.

**Critical Note:** We were cheating a bit here! There is a stochastic component and if one re-reruns the cells results will differ. In fact, *in this simple case* the last configuration is often the best.

### 4. Splitting Models - Multiple Outputs

Now, let's go back to our dataset and let's try to predict two labels. We want to model that by *splitting* the model, i.e., two distinct hidden layers act on the input, and we use those two branches to predict the two labels:  

In [14]:
# Define Input layer(s)
input_numbers = tf.keras.layers.Input(shape=(10,), name="input_numbers")

# Define separate hidden layers and acting on (same!) input
dense_layer_1 = tf.keras.layers.Dense(10, activation='relu', name='dense_1')
dense_layer_2 = tf.keras.layers.Dense(10, activation='relu', name='dense_2')

dense_output_1 = dense_layer_1(input_numbers)     # hideen layer 1 acting on input
dense_output_2 = dense_layer_2(input_numbers)     # hideen layer 2 acting on input



# Define classification layer and act on previous output to classify examples
classification_layer_1 = tf.keras.layers.Dense(1, 
                            activation='sigmoid', name='classification_1')  # classification layer 1 for first branch

classification_layer_2 = tf.keras.layers.Dense(1, 
                            activation='sigmoid', name='classification_2')  # classification layer 2 for second branch


classification_output_1 = classification_layer_1(dense_output_1)
classification_output_2 = classification_layer_2(dense_output_2)



# Build and compile model

model_input = input_numbers
model_output = [classification_output_1, classification_output_2]
losses = ['binary_crossentropy', 'binary_crossentropy']
metrics = ['acc', 'acc']

model = tf.keras.models.Model(model_input, model_output)
model.compile(loss=losses,  optimizer='adam', metrics=metrics)




In [15]:
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_numbers (InputLayer)      [(None, 10)]         0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 10)           110         input_numbers[0][0]              
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 10)           110         input_numbers[0][0]              
__________________________________________________________________________________________________
classification_1 (Dense)        (None, 1)            11          dense_1[0][0]                    
____________________________________________________________________________________________

In [16]:
model.fit(
    train_X,
    [train_a_y, train_b_y],
    validation_data=[test_X, [test_a_y, test_b_y]],
    epochs=150,
    verbose=1
)

Train on 400 samples, validate on 400 samples
Epoch 1/150
400/400 [==============================] - 0s 1ms/sample - loss: 1.5079 - classification_1_loss: 0.7331 - classification_2_loss: 0.7740 - classification_1_acc: 0.4375 - classification_2_acc: 0.4925 - val_loss: 1.4894 - val_classification_1_loss: 0.7307 - val_classification_2_loss: 0.7637 - val_classification_1_acc: 0.4275 - val_classification_2_acc: 0.4825
Epoch 2/150
400/400 [==============================] - 0s 92us/sample - loss: 1.4619 - classification_1_loss: 0.7168 - classification_2_loss: 0.7457 - classification_1_acc: 0.4525 - classification_2_acc: 0.4900 - val_loss: 1.4480 - val_classification_1_loss: 0.7147 - val_classification_2_loss: 0.7297 - val_classification_1_acc: 0.4500 - val_classification_2_acc: 0.4925
Epoch 3/150
400/400 [==============================] - 0s 88us/sample - loss: 1.4241 - classification_1_loss: 0.7034 - classification_2_loss: 0.7202 - classification_1_acc: 0.4875 - classification_2_acc: 0.4925 

Epoch 23/150
400/400 [==============================] - 0s 91us/sample - loss: 1.1222 - classification_1_loss: 0.5245 - classification_2_loss: 0.5966 - classification_1_acc: 0.8725 - classification_2_acc: 0.6875 - val_loss: 1.1407 - val_classification_1_loss: 0.5202 - val_classification_2_loss: 0.6212 - val_classification_1_acc: 0.8725 - val_classification_2_acc: 0.6525
Epoch 24/150
400/400 [==============================] - 0s 92us/sample - loss: 1.1066 - classification_1_loss: 0.5126 - classification_2_loss: 0.5885 - classification_1_acc: 0.8725 - classification_2_acc: 0.6850 - val_loss: 1.1259 - val_classification_1_loss: 0.5066 - val_classification_2_loss: 0.6160 - val_classification_1_acc: 0.8775 - val_classification_2_acc: 0.6575
Epoch 25/150
400/400 [==============================] - 0s 89us/sample - loss: 1.0913 - classification_1_loss: 0.5087 - classification_2_loss: 0.5897 - classification_1_acc: 0.8700 - classification_2_acc: 0.6825 - val_loss: 1.1115 - val_classification_1_

Epoch 45/150
400/400 [==============================] - 0s 88us/sample - loss: 0.8508 - classification_1_loss: 0.3383 - classification_2_loss: 0.5096 - classification_1_acc: 0.8775 - classification_2_acc: 0.7500 - val_loss: 0.8834 - val_classification_1_loss: 0.3204 - val_classification_2_loss: 0.5586 - val_classification_1_acc: 0.9050 - val_classification_2_acc: 0.7225
Epoch 46/150
400/400 [==============================] - 0s 88us/sample - loss: 0.8424 - classification_1_loss: 0.3345 - classification_2_loss: 0.5052 - classification_1_acc: 0.8800 - classification_2_acc: 0.7475 - val_loss: 0.8761 - val_classification_1_loss: 0.3219 - val_classification_2_loss: 0.5539 - val_classification_1_acc: 0.9100 - val_classification_2_acc: 0.7225
Epoch 47/150
400/400 [==============================] - 0s 86us/sample - loss: 0.8344 - classification_1_loss: 0.3281 - classification_2_loss: 0.5031 - classification_1_acc: 0.8800 - classification_2_acc: 0.7475 - val_loss: 0.8679 - val_classification_1_

Epoch 67/150
400/400 [==============================] - 0s 90us/sample - loss: 0.7198 - classification_1_loss: 0.2585 - classification_2_loss: 0.4605 - classification_1_acc: 0.8950 - classification_2_acc: 0.7800 - val_loss: 0.7551 - val_classification_1_loss: 0.2365 - val_classification_2_loss: 0.5251 - val_classification_1_acc: 0.9375 - val_classification_2_acc: 0.7400
Epoch 68/150
400/400 [==============================] - 0s 95us/sample - loss: 0.7148 - classification_1_loss: 0.2647 - classification_2_loss: 0.4562 - classification_1_acc: 0.8950 - classification_2_acc: 0.7800 - val_loss: 0.7506 - val_classification_1_loss: 0.2319 - val_classification_2_loss: 0.5171 - val_classification_1_acc: 0.9375 - val_classification_2_acc: 0.7450
Epoch 69/150
400/400 [==============================] - 0s 89us/sample - loss: 0.7120 - classification_1_loss: 0.2603 - classification_2_loss: 0.4569 - classification_1_acc: 0.9000 - classification_2_acc: 0.7800 - val_loss: 0.7468 - val_classification_1_

Epoch 89/150
400/400 [==============================] - 0s 89us/sample - loss: 0.6481 - classification_1_loss: 0.2146 - classification_2_loss: 0.4246 - classification_1_acc: 0.9150 - classification_2_acc: 0.8025 - val_loss: 0.6858 - val_classification_1_loss: 0.1887 - val_classification_2_loss: 0.4910 - val_classification_1_acc: 0.9475 - val_classification_2_acc: 0.7575
Epoch 90/150
400/400 [==============================] - 0s 90us/sample - loss: 0.6465 - classification_1_loss: 0.2121 - classification_2_loss: 0.4312 - classification_1_acc: 0.9125 - classification_2_acc: 0.8050 - val_loss: 0.6834 - val_classification_1_loss: 0.1894 - val_classification_2_loss: 0.4992 - val_classification_1_acc: 0.9500 - val_classification_2_acc: 0.7550
Epoch 91/150
400/400 [==============================] - 0s 90us/sample - loss: 0.6444 - classification_1_loss: 0.2121 - classification_2_loss: 0.4306 - classification_1_acc: 0.9150 - classification_2_acc: 0.7975 - val_loss: 0.6801 - val_classification_1_

Epoch 111/150
400/400 [==============================] - 0s 101us/sample - loss: 0.6072 - classification_1_loss: 0.1923 - classification_2_loss: 0.4111 - classification_1_acc: 0.9225 - classification_2_acc: 0.8100 - val_loss: 0.6462 - val_classification_1_loss: 0.1631 - val_classification_2_loss: 0.4909 - val_classification_1_acc: 0.9500 - val_classification_2_acc: 0.7650
Epoch 112/150
400/400 [==============================] - 0s 108us/sample - loss: 0.6063 - classification_1_loss: 0.1898 - classification_2_loss: 0.4073 - classification_1_acc: 0.9225 - classification_2_acc: 0.8100 - val_loss: 0.6452 - val_classification_1_loss: 0.1598 - val_classification_2_loss: 0.4882 - val_classification_1_acc: 0.9550 - val_classification_2_acc: 0.7650
Epoch 113/150
400/400 [==============================] - 0s 96us/sample - loss: 0.6046 - classification_1_loss: 0.1958 - classification_2_loss: 0.4087 - classification_1_acc: 0.9225 - classification_2_acc: 0.8125 - val_loss: 0.6431 - val_classificati

Epoch 133/150
400/400 [==============================] - 0s 96us/sample - loss: 0.5834 - classification_1_loss: 0.1788 - classification_2_loss: 0.3994 - classification_1_acc: 0.9275 - classification_2_acc: 0.8175 - val_loss: 0.6247 - val_classification_1_loss: 0.1485 - val_classification_2_loss: 0.4849 - val_classification_1_acc: 0.9575 - val_classification_2_acc: 0.7675
Epoch 134/150
400/400 [==============================] - 0s 98us/sample - loss: 0.5825 - classification_1_loss: 0.1789 - classification_2_loss: 0.3969 - classification_1_acc: 0.9250 - classification_2_acc: 0.8150 - val_loss: 0.6235 - val_classification_1_loss: 0.1490 - val_classification_2_loss: 0.4683 - val_classification_1_acc: 0.9550 - val_classification_2_acc: 0.7700
Epoch 135/150
400/400 [==============================] - 0s 96us/sample - loss: 0.5816 - classification_1_loss: 0.1829 - classification_2_loss: 0.3995 - classification_1_acc: 0.9250 - classification_2_acc: 0.8200 - val_loss: 0.6228 - val_classification

**Questions:**
* Does it surprise you that the loss for label 1 is smaller than the loss for label 2?    
* Quick side check... do we overtrain?    

### 5. Custom Layers

Let's define a simple custom layer which is nothing but an affine transformation followed by a relu activation. We need to define:
* \_\_init\_\_()      
* build()   
* call()   
* compute_output_shape()   

In [17]:
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Layer

class MyStupidDenseLayer(Layer):

    def __init__(self, output_dim, **kwargs):   
        self.output_dim = output_dim
        
        super(MyStupidDenseLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.W = self.add_variable("W",
                                    shape=[int(input_shape[1]),self.output_dim], 
                                   initializer=tf.glorot_normal_initializer(),
                                   trainable=True)
        self.b = self.add_variable("b",
                                    shape=[self.output_dim, ], 
                                   initializer=tf.initializers.zeros(),
                                   trainable=True)
        super(MyStupidDenseLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, x):
        return tf.nn.relu(tf.tensordot(x, self.W, axes=[[1],[0]]) + self.b)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

Let's use our custom layer in the same architecture. 

In [18]:
# Define Input layer(s)
input_numbers = tf.keras.layers.Input(shape=(10,), name="input_numbers")

# Define separate hidden layers and acting on (same!) input
dense_layer_1 = MyStupidDenseLayer(10, name='dense_1')
dense_layer_2 = MyStupidDenseLayer(10, name='dense_2')

dense_output_1 = dense_layer_1(input_numbers)
dense_output_2 = dense_layer_2(input_numbers)



# Define classification layer and act on previous output to classify examples
classification_layer_1 = tf.keras.layers.Dense(1, 
                            activation='sigmoid', name='classification_1')

classification_layer_2 = tf.keras.layers.Dense(1, 
                            activation='sigmoid', name='classification_2')


classification_output_1 = classification_layer_1(dense_output_1)
classification_output_2 = classification_layer_2(dense_output_2)



# Build and compile model

model_input = input_numbers
model_output = [classification_output_1, classification_output_2]
losses = ['binary_crossentropy', 'binary_crossentropy']
metrics = ['acc', 'acc']

model = tf.keras.models.Model(model_input, model_output)
model.compile(loss=losses,  optimizer='adam', metrics=metrics)




Instructions for updating:
Please use `layer.add_weight` method instead.


In [19]:
model.fit(
    train_X,
    [train_a_y, train_b_y],
    validation_data=[test_X, [test_a_y, test_b_y]],
    epochs=149,
    verbose=0
)

model.fit(
    train_X,
    [train_a_y, train_b_y],
    validation_data=[test_X, [test_a_y, test_b_y]],
    epochs=1,
    verbose=1
)

Train on 400 samples, validate on 400 samples
400/400 [==============================] - 0s 95us/sample - loss: 0.5857 - classification_1_loss: 0.1766 - classification_2_loss: 0.4084 - classification_1_acc: 0.9325 - classification_2_acc: 0.7950 - val_loss: 0.6058 - val_classification_1_loss: 0.1326 - val_classification_2_loss: 0.4780 - val_classification_1_acc: 0.9625 - val_classification_2_acc: 0.7725


Close enough. So the custom layer works exactly as expected.


### 6. Sharing & Freezing Weights 


Let's define an identical model. The weights for this model will be re-initialized, so upon training for 1 epoch the loss will be much higher than after 150 epochs for the original model: 

In [20]:
# Define Input layer(s)
input_numbers = tf.keras.layers.Input(shape=(10,), name="input_numbers")

# Define separate hidden layers and acting on (same!) input
dense_layer_1 = MyStupidDenseLayer(10, name='dense_1')
dense_layer_2 = MyStupidDenseLayer(10, name='dense_2')

dense_output_1 = dense_layer_1(input_numbers)
dense_output_2 = dense_layer_2(input_numbers)



# Define classification layer and act on previous output to classify examples
classification_layer_1 = tf.keras.layers.Dense(1, 
                            activation='sigmoid', name='classification_1')

classification_layer_2 = tf.keras.layers.Dense(1, 
                            activation='sigmoid', name='classification_2')


classification_output_1 = classification_layer_1(dense_output_1)
classification_output_2 = classification_layer_2(dense_output_2)



# Build and compile model

model_input = input_numbers
model_output = [classification_output_1, classification_output_2]
losses = ['binary_crossentropy', 'binary_crossentropy']
metrics = ['acc', 'acc']

model_mirror = tf.keras.models.Model(model_input, model_output)
model_mirror.compile(loss=losses,  optimizer='adam', metrics=metrics)




In [21]:
model_mirror.fit(
    train_X,
    [train_a_y, train_b_y],
    validation_data=[test_X, [test_a_y, test_b_y]],
    epochs=1,
    verbose=1
)

Train on 400 samples, validate on 400 samples
400/400 [==============================] - 0s 1ms/sample - loss: 1.4511 - classification_1_loss: 0.7201 - classification_2_loss: 0.7294 - classification_1_acc: 0.4925 - classification_2_acc: 0.4475 - val_loss: 1.4177 - val_classification_1_loss: 0.7029 - val_classification_2_loss: 0.7156 - val_classification_1_acc: 0.4950 - val_classification_2_acc: 0.4425


Indeed, this model would make very different predictions - the loss is ~1.4 vs ~0.6. No surprise, obviously, as this model has not been trained.

What if we manually set the the weights of the second model to the weights of the first? Let's look at the layers for both models and then copy the weights:

In [22]:
model.layers

Let's look at one layer in more detail:

In [23]:
len(model.layers[1].get_weights())

2

Why would that be? No magic.. weights and biases of layer!

In [24]:
np.shape(model.layers[1].get_weights()[0])

(10, 10)

In [25]:
np.shape(model.layers[1].get_weights()[1])

(10,)

Cool. What is the name of the layer?

In [26]:
model.layers[1].name

'dense_1'

Correct. Is it trainable?

In [27]:
model.layers[1].trainable

True

Great. Are the layers of the second model essentially the same?

In [28]:
model_mirror.layers

Yup. (We could look at shapes, etc, but won't do that now.) Now we set the second model's weights to equal the first model's trained weights:

In [29]:
model_mirror.layers[1].set_weights(model.layers[1].get_weights())
model_mirror.layers[2].set_weights(model.layers[2].get_weights())
model_mirror.layers[3].set_weights(model.layers[3].get_weights())
model_mirror.layers[4].set_weights(model.layers[4].get_weights())

**Question:** Why don't we set the weights for layer 0?

Okay. What is the loss now for the mirror model? 

In [30]:
model_mirror.fit(
    train_X,
    [train_a_y, train_b_y],
    validation_data=[test_X, [test_a_y, test_b_y]],
    epochs=1,
    verbose=1
)

Train on 400 samples, validate on 400 samples
400/400 [==============================] - 0s 99us/sample - loss: 0.5866 - classification_1_loss: 0.1706 - classification_2_loss: 0.4157 - classification_1_acc: 0.9300 - classification_2_acc: 0.7900 - val_loss: 0.6071 - val_classification_1_loss: 0.1360 - val_classification_2_loss: 0.4744 - val_classification_1_acc: 0.9625 - val_classification_2_acc: 0.7675


Cool! Essentially the same as for the original one! (Not a surprise actually... but great that it works.)

Let's now turn to **freezing** layers. For example, you may want to copy weights from another model but hold those weights fixed upon further training. To test this, we will do a simple toy exercise: freeze a layer of the second model and train it further. Let's compare some weights before and after training. 

We start by defining a new model that again has the same architecture, **but we set the trainable-parameter for one layer to False**: 

In [31]:
# Define Input layer(s)
input_numbers = tf.keras.layers.Input(shape=(10,), name="input_numbers")

# Define separate hidden layers and acting on (same!) input
dense_layer_1 = MyStupidDenseLayer(10, name='dense_1')

dense_layer_2 = MyStupidDenseLayer(10, name='dense_2')
dense_layer_2.trainable = False                                   # Freeze this layer

dense_output_1 = dense_layer_1(input_numbers)
dense_output_2 = dense_layer_2(input_numbers)



# Define classification layer and act on previous output to classify examples
classification_layer_1 = tf.keras.layers.Dense(1, 
                            activation='sigmoid', name='classification_1')

classification_layer_2 = tf.keras.layers.Dense(1, 
                            activation='sigmoid', name='classification_2')



classification_output_1 = classification_layer_1(dense_output_1)
classification_output_2 = classification_layer_2(dense_output_2)



# Build and compile model

model_input = input_numbers
model_output = [classification_output_1, classification_output_2]
losses = ['binary_crossentropy', 'binary_crossentropy']
metrics = ['acc', 'acc']

model_mirror_2 = tf.keras.models.Model(model_input, model_output)
model_mirror_2.compile(loss=losses,  optimizer='adam', metrics=metrics)


Next, we again set the weights to mirror the first model:

In [32]:
model_mirror_2.layers[1].set_weights(model.layers[1].get_weights())
model_mirror_2.layers[2].set_weights(model.layers[2].get_weights())
model_mirror_2.layers[3].set_weights(model.layers[3].get_weights())
model_mirror_2.layers[4].set_weights(model.layers[4].get_weights())

What are (some of) the weights now, before we train further?

In [33]:
model_mirror_2.layers[1].get_weights()[0][:3, :3]

array([[ 0.5493507 ,  0.64608824, -0.4960204 ],
       [ 0.5602562 ,  0.54325235, -0.7760406 ],
       [ 0.33898988,  0.8414829 ,  0.03868393]], dtype=float32)

In [34]:
model_mirror_2.layers[2].get_weights()[0][:3, :3]

array([[ 0.03614025, -0.07353805,  0.03746113],
       [-0.06931327, -0.41153473, -0.25313395],
       [-0.4576991 , -0.01952373,  0.2951088 ]], dtype=float32)

We will compare that with the values post further training. Let's also check the 'trainable' settings:

In [35]:
model_mirror_2.layers[1].trainable

True

In [36]:
model_mirror_2.layers[2].trainable

False

Good, so it shows that one hidden layer supposedly is trainable, the other one is not. Is that true? We train more and compare weights compared to what they were:

In [37]:
model_mirror_2.fit(
    train_X,
    [train_a_y, train_b_y],
    validation_data=[test_X, [test_a_y, test_b_y]],
    epochs=40,
    verbose=0
)
model_mirror_2.fit(
    train_X,
    [train_a_y, train_b_y],
    validation_data=[test_X, [test_a_y, test_b_y]],
    epochs=1,
    verbose=1
)

Train on 400 samples, validate on 400 samples
400/400 [==============================] - 0s 97us/sample - loss: 0.5713 - classification_1_loss: 0.1696 - classification_2_loss: 0.4112 - classification_1_acc: 0.9350 - classification_2_acc: 0.7950 - val_loss: 0.5972 - val_classification_1_loss: 0.1281 - val_classification_2_loss: 0.4683 - val_classification_1_acc: 0.9725 - val_classification_2_acc: 0.7625


In [38]:
model_mirror_2.layers[1].get_weights()[0][:3, :3]

array([[ 0.5515234 ,  0.648011  , -0.5317562 ],
       [ 0.5594    ,  0.5445362 , -0.80385584],
       [ 0.34686086,  0.8477436 ,  0.02895916]], dtype=float32)

These weights **did change**.

In [39]:
model_mirror_2.layers[2].get_weights()[0][:3, :3]

array([[ 0.03614025, -0.07353805,  0.03746113],
       [-0.06931327, -0.41153473, -0.25313395],
       [-0.4576991 , -0.01952373,  0.2951088 ]], dtype=float32)

These weights **did not change**. So here we could see layer freezing at work!